LASSO

In [1]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bin
import time
#As an example for linear case
# patiences = 10
# n_node = 50
# n_layer = 1
# t_nodes = 100
# t_fig = 50
# n_lr = 1e-3
# n_epoch = 1000
# s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')    
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    St = E_lasso_bin(train_data, val_data, test_data,lam_min=-2,lam_max=0,K=5)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.06706000000000001
0.02476845574516102
0.005339636806258849
0.0047401885146060195


In [2]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    St = E_lasso_bin(train_data, val_data, test_data,lam_min=-2,lam_max=0,K=5)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.30820000000000003
0.013941305534274768
0.09605397628982569
0.010949715656512283


RDL

In [3]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep 
from log_dlearn_binary import RDL_bin
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')    
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    res_rdl = RDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10))
    d_est = res_rdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.07277
0.02540289550425305
0.0065393590626161725
0.005912759326814371


In [4]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep 
from log_dlearn_binary import RDL_bin
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    res_rdl = RDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10))
    d_est = res_rdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))

    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.32302
0.02153414962333085
0.10628119985536187
0.014241978282866892


SDL

In [5]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep 
from log_dlearn_binary import SDL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    res_sdl = SDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.44563
0.11957910812512361
0.1883543411724701
0.07566525198449789


In [6]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep 
from log_dlearn_binary import SDL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')  
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    res_sdl = SDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.33025
0.045272922370883026
0.10905835065696533
0.029802058857899716


DL

In [1]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep 
from log_dlearn_binary import DL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    res_sdl = DL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.08675000000000001
0.03976251375353425
0.01100910536872909
0.01159468583879227


In [2]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep 
from log_dlearn_binary import DL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')  
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    res_sdl = DL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.32417
0.04741678078486561
0.10759595400563228
0.03131474725289006


OLS

In [7]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from optITR import optITR_lin_bin
import time
#As an example for linear case
# patiences = 10
# n_node = 50
# n_layer = 1
# t_nodes = 100
# t_fig = 50
# n_lr = 1e-3
# n_epoch = 1000
# s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    St = E_ols_bin(train_data, val_data, test_data)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.05872000000000001
0.022383065026934983
0.004881368444059511
0.004530764262280522


In [8]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    St = E_ols_bin(train_data, val_data, test_data)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.31092
0.015732564952988443
0.09666052265792296
0.010141407953605845


DLS

In [9]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_dls_binary import E_dls_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200
patiences = 20
n_node = 64
n_layer = 2
n_lr = 1.3e-3
n_epoch = 1000
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
test_data = gendata_Linear(500, 0.5,beta,gamma)
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    St = E_dls_bin(train_data, val_data, test_data, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat,beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.10981
0.03415646205332162
0.016917306129972897
0.010556528106559151


In [10]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Deep
import torch
from log_dls_binary import E_dls_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200
patiences = 20
n_node = 64
n_layer = 2
n_lr = 1.3e-3
n_epoch = 1000
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    St = E_dls_bin(train_data, val_data, test_data, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.20622
0.09064453430847334
0.060688040198328116
0.046730659760852465


Proposed

In [ ]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_hazard_binary import g_dnn_bin
from optITR import optITR_lin_bin
import time
#As an example for linear case
patiences = 10
n_node = 50
n_layer = 1
t_nodes = 100
t_fig = 50
n_lr = 2e-4
n_epoch = 1000
s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear(100, 0.5,beta,gamma)
    train_data = gendata_Linear(400, 0.5,beta,gamma)
    St = g_dnn_bin(train_data, val_data, test_data, t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    S_R_X_ibs = St['S_R_X_untreat']
    S_R_X_ibs1 = St['S_R_X_treated']
    Lambda_R_X_untreat = St['Lambda_R_X_untreat']
    Lambda_R_X_treated = St['Lambda_R_X_treated']
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.10912000000000001
0.06825881335036524
0.01902237279807937
0.037375396967145945


In [12]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
import torch
from log_hazard_binary import g_dnn_bin
from optITR import optITR_deep_bin
import time
#As an example for linear case
patiences = 20
n_node = 64
n_layer = 1
t_nodes = 100
t_fig = 50
n_lr = 1.2e-3
n_epoch = 1000
s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep(100, 0.5)
    train_data = gendata_Deep(400, 0.5)
    St = g_dnn_bin(train_data, val_data, test_data, t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    S_R_X_ibs = St['S_R_X_untreat']
    S_R_X_ibs1 = St['S_R_X_treated']
    Lambda_R_X_untreat = St['Lambda_R_X_untreat']
    Lambda_R_X_treated = St['Lambda_R_X_treated']
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.1882
0.04896161762033604
0.04728157942074852
0.01888605744858731
